In [ ]:
import yaml
import time
from datetime import datetime
from megadetector.detection import run_detector
import pandas as pd

# Adjust the path as needed to point to the directory containing alert_system_utils.py
import sys
sys.path.append('../scripts') 

from alert_system_utils import (
    detector,
    classifier,
    set_device,
    check_emails,
    extract_and_update_camera_info,
    update_camera_data_dataframe,
    batch_classification,
    current_time,
    generate_alert_caption,
    valid_detections,
    get_current_time_in_romania,
    send_alert_to_telegram,
    save_images,
    annotate_images
)

In [ ]:
# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']
TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID = config['telegram_config']['chat_id'] # '-1002249589791' # replace with config after tests

# Detection and Classification Model Settings
DETECTOR_MODEL_PATH = '../models/md_v5a.0.0.pt'
DETECTOR_CLASSES = ["animal", "human", "vehicle"]
DETECTION_THRESHOLD = 0.10

BACKBONE = 'vit_large_patch14_dinov2'
CLASSIFIER_MODEL_PATH = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'
CLASSIFIER_CLASSES = ["Badger", "Ibex", "Red Deer", "Chamois", "Cat", "Goat", "Roe Deer", "Dog", "Squirrel", "Equid", "Genet", "Hedgehog", "Lagomorph", "Wolf", "Lynx", "Marmot", "Micromammal", "Mouflon", "Sheep", "Mustelid", "Bird", "Bear", "Nutria", "Fox", "Wild Boar", "Cow"]
SPECIES_OF_INTEREST = ["Wild Boar", "Bear"]
CLASSIFICATION_THRESHOLD = 0.20

CAPTURE_DATABASE_PATH = '../data/capture_database.csv'
CAMERA_LOCATIONS_PATH = '../data/camera_locations.csv'
PHOTOS_PATH = '../data/photos/'
ALERT_LANGUAGE = "en" # Enter 'en' for English, 'ro' for Romanian
HUMAN_ALERT_START = "21:00"
HUMAN_ALERT_END = "06:00"

# Initialise the Detection and Classifier Models
device = set_device()
detector_model = run_detector.load_detector(DETECTOR_MODEL_PATH)
print("Loading classifier...")
start_time = time.time()
classifier_model = classifier(CLASSIFIER_MODEL_PATH, BACKBONE, CLASSIFIER_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

In [ ]:
from PIL import Image, ImageDraw
import io

def annotate_images(df, images):
    image_list = []
    
    # Extract the last len(images) rows
    last_rows = df.iloc[-len(images):]

    for image, row in zip(images, last_rows.iterrows()):
        index, row_data = row
        
        # Resize the image to 800x600
        image = image.resize((800, 600), Image.LANCZOS)
        
        draw = ImageDraw.Draw(image)

        detection_boxes = row_data['Detection Boxes']
        detection_classes = row_data['Detection Classes']
        detection_confidences = row_data['Detection Confidences']
        species_classes = row_data['Species Classes']
        species_confidences = row_data['Species Confidences']
        
        species_idx = 0  # to keep track of the current species index

        for box, d_class, d_conf in zip(detection_boxes, detection_classes, detection_confidences):
            # Convert relative coordinates to absolute coordinates
            width, height = image.size
            x1 = int(box[0] * width)
            y1 = int(box[1] * height)
            x2 = int((box[0] + box[2]) * width)
            y2 = int((box[1] + box[3]) * height)
            
            # Set color and label based on detection class
            if d_class == '1':  # Animal
                color = 'red'
                label = f"{species_classes[species_idx]} {species_confidences[species_idx] * 100:.0f}%"
                species_idx += 1
            elif d_class == '2':  # Human
                color = 'green'
                label = f"Human {d_conf * 100:.0f}%"
            elif d_class == '3':  # Vehicle
                color = 'blue'
                label = f"Vehicle {d_conf * 100:.0f}%"
            else:
                color = 'yellow'
                label = f"Unknown {d_conf * 100:.0f}%"
                
            # Draw bounding box
            draw.rectangle([x1, y1, x2, y2], outline=color, width=3)
            draw.text((x1, y1 - 10), label, fill=color)
        
        image_list.append(image)
    
    return image_list


In [ ]:
if __name__ == "__main__":
    print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            images, original_images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            
            # TO DO: delete emails older than 48 hours
            # Delete human images saved on disk older than 48 hours

            if camera_id:

                camera_make, gps, location, map_url, battery, sd_memory = extract_and_update_camera_info(CAMERA_LOCATIONS_PATH, camera_id, battery, sd_memory)
                
                # TO DO: if camera_id is UOVision, wait 10, check for another, then append or continue.
                # check subject to determine if uovision, then only open if it is

                df = pd.read_csv(CAPTURE_DATABASE_PATH)
                df = update_camera_data_dataframe(df, len(images), camera_id, camera_make, img_date, img_time, temp_deg_c, battery, sd_memory, location, gps, map_url)

                if images:

                    df = detector(df, detector_model, images, DETECTION_THRESHOLD)
                    
                    df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD) # test classification threshold

                    # TO DO: IN A BATCH OF MOSTLY EMPTY PHOTOS, IF THERE IS ONE DETECTION, PRIMARY SPECIES IS STILL EMPTY

                    if valid_detections(df, images):

                        df, alert_caption, human_warning, priority_alert = generate_alert_caption(df, len(images), SPECIES_OF_INTEREST, EMAIL_USER, ALERT_LANGUAGE)

                        current_time_in_romania = get_current_time_in_romania()

                        if human_warning:
                            if HUMAN_ALERT_START <= current_time_in_romania or current_time_in_romania < HUMAN_ALERT_END:
                                print(f"{current_time()} | Time in Romania: {current_time_in_romania} - sending human photos")
                                alert_images = annotate_images(df, images)
                            else:
                                print(f"{current_time()} | Time in Romania: {current_time_in_romania} - human photos NOT sent")
                                alert_images = None
                        else:
                            print(f"{current_time()} | No human warning - sending photos regardless of time")
                            alert_images = annotate_images(df, images)


                        send_alert_to_telegram(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, alert_images, alert_caption)

                        #print(f"{alert_caption}")
                    else:

                        print(f"{current_time()} | Empty photos")

                    df = save_images(df, original_images, PHOTOS_PATH)

                else:
                    print(f"{current_time()} | No images attached to email")
                
                df.to_csv(CAPTURE_DATABASE_PATH, index=False)
                print(f"{current_time()} | Capture database updated: {CAPTURE_DATABASE_PATH}")
                del df

            else:
                time.sleep(10)

        except KeyboardInterrupt:
            print(f"{current_time()} | Interrupted by user")
            break

        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(10)
            print(f"\nMonitoring {EMAIL_USER} for new messages...")
            continue
